In [2]:
%load_ext autoreload
%autoreload 2
from utils import*
from preprocessing import preprocessing
from feature_extraction import extract_features
from feature_extraction import cluster_descriptors
from training import train_svm
from sklearn.cluster import KMeans

Image Preprocessing

In [ ]:
for i in range(0, 6):
    print(i)
    path_of_the_directory='../data/men/'+str(i)+'/'
    path_of_the_directory_result='../preprocessing_output/men/'+str(i)+'/'
    ind=0
    for filename in os.listdir(path_of_the_directory):
        if(ind>20):
            break
        ind=ind+1
        img = cv2.imread(path_of_the_directory+str(filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preprocessing(img, gamma=True, close=True)
        cv2.imwrite(path_of_the_directory_result+filename,binary_hand)

    path_of_the_directory='../data/women/'+str(i)+'/'
    path_of_the_directory_result='../preprocessing_output/women/'+str(i)+'/'
    ind=0
    for filename in os.listdir(path_of_the_directory):
        if(ind>20):
            break
        ind=ind+1
        img = cv2.imread(path_of_the_directory+str( filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preprocessing(img, gamma=True, close=True)
        cv2.imwrite(path_of_the_directory_result+filename,binary_hand)
    

Feature Extraction

In [99]:
feature_vector_men = []
expected_class_men = []
feature_vector_women = []
expected_class_women = []
for i in range(0, 2):
    print(i)
    path_of_the_directory_result_men = '../preprocessing_output/men/' + str(i) + '/'
    path_of_the_directory_result_women = '../preprocessing_output/women/' + str(i) + '/'

    for filename in os.listdir(path_of_the_directory_result_men):
        img = cv2.imread(path_of_the_directory_result_men+str(filename), cv2.IMREAD_GRAYSCALE)
        # print(path_of_the_directory_result_men + str(filename))
        # corners_detected = detectCorners(path_of_the_directory_result_men + str(filename))
        corners_descriptors=extract_features(img)
        feature_vector_men.append(corners_descriptors)
        expected_class_men.append(i)
    
    # export feature_vector_men to excel

    # for filename in os.listdir(path_of_the_directory_result_women):
    #     img = cv2.imread(path_of_the_directory_result_women+str(filename), cv2.IMREAD_GRAYSCALE)
    #     # print(path_of_the_directory_result_women + str(filename))
    #     # corners_detected = detectCorners(path_of_the_directory_result_women + str(filename))
    #     # feature_vector_women.append(corners_detected)
    #     corners_descriptors=extract_features(img)
    #     feature_vector_women.append(corners_descriptors)
    #     expected_class_women.append(i)
    
    # export feature_vector_women to excel

print(path_of_the_directory_result_men)

0
1
../preprocessing_output/men/1/


In [102]:
# print(np.shape(feature_vector_men))
# print(expected_class_men)
# y = []
# for fvm in feature_vector_men: 
#     # y = np.hstack(np.full(len(fvm),expected_class_men[i]) for i in fvm)
#     y.append(np.vstack([expected_class_men[i]]*len(fvm) for i in fvm))
# feature_vector_men = feature_vector_men.flatten()

# feature_vector_men = np.asarray(feature_vector_men)
# feature_vector_men.flatten()

# for k in feature_vector_men:
#     if(k is None):
#         print(0)
#     else:
#         print(len(k))
#         for j in k:
#             if(j is None):
#                 print(0, end=',')
#             else:
#                 print(len(j), end=',')
#         print()

# Original code
# descriptors = []
# for k in feature_vector_men:
#     if(k is not None):
#         for j in k:
#             descriptors.append(j)
descriptors = [j for k in feature_vector_men if k is not None for j in k]   #Cleaner version
print(len(descriptors))
num_of_clusters = 3
num_of_images = len(feature_vector_men)
kmeans = KMeans(n_clusters=num_of_clusters).fit(descriptors)
final_fv = np.zeros((num_of_images, num_of_clusters))
k_indx = 0
for img_indx in range(len(feature_vector_men)):
    if feature_vector_men[img_indx] is not None:
        for p in range(len(feature_vector_men[img_indx])):
            final_fv[img_indx][kmeans.labels_[k_indx]] += 1
            k_indx += 1
# print(expected_class_men)
predicted_men, test = train_svm(final_fv, expected_class_men)
print(predicted_men)
print(test)
# predicted_women = train_svm(feature_vector_women, expected_class_women)

18
[0 0 0 0 0 0 0 0 0]
[1, 0, 0, 1, 0, 1, 0, 1, 1]


c:\Users\mohab\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cleaner version of feature extraction

In [5]:
feature_vector_men = []
expected_class_men = []

for i in range(0, 1):
    print(i)
    path_of_the_directory_result_men = '../preprocessing_output/men/' + str(i) + '/'
    feature_descriptors, expexted_classes = extract_features(path_of_the_directory=path_of_the_directory_result_men, classification=i)
    # print(feature_descriptors)
    # print(expexted_classes)

    for fd in feature_descriptors:
        feature_vector_men.append(fd)
    for ec in expexted_classes:
        expected_class_men.append(ec)


0


Model Training

In [6]:
feature_vector_men = cluster_descriptors(feature_vector_men, expected_class_men)
print(feature_vector_men)
# prediction, actual = train_svm(feature_vector_men, expected_class_men)
# print(prediction)
# print(actual)

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[[1. 2. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
